In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import math
import os
import copy
import time
import PIL
import torch.utils.data as Data
from torchvision import transforms, models
from typing import Any, Callable, List, Optional, Union, Tuple
from torchvision.datasets.vision import VisionDataset


class RanzcrDataset(VisionDataset):
    def __init__(
        self,
        root: str,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,):

        super(RanzcrDataset, self).__init__(root, transform=transform,
                                     target_transform=target_transform)    

        self.train_data = pd.read_csv(os.path.join(self.root, 'ranzcr-clip-catheter-line-classification', "sample_submission.csv"),  header=0)
        self.train_data = self.train_data.values
       
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        fn = self.train_data[index, 0] + '.jpg'
        X = PIL.Image.open(os.path.join(self.root, 'ranzcr-clip-catheter-line-classification/test', fn))
        X=X.convert('RGB')
        X = X.resize((100, 100), PIL.Image.BILINEAR)
        X_tensor=transforms.ToTensor()(X)
        X_tensor=transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(X_tensor)
              
        target = self.train_data[index, 1:-2]
        target_tensor = torch.tensor(target.tolist())
        return X_tensor, target_tensor

    def __len__(self) -> int:
        return len(self.train_data)


def valid_model(model_list, validdataloader, targetCol, criterion):
    prediction = torch.tensor([]).to("cuda")
    for images, labels in validdataloader:
        b_x = images.to("cuda")

        stack = torch.tensor([]).to("cuda")
        for model in model_list:
            model.eval()
            output = model(b_x)
            new = torch.argmax(output,1).unsqueeze(0)
            # print("new: ", new.shape)
            stack = torch.cat((stack, new),0)

            # print("Stack: ",stack.shape)
        prediction = torch.cat((prediction, stack), 1)
        print("***Prediction***", prediction.shape)
    return torch.Tensor.numpy(prediction.cpu())


class RanZcrNet(nn.Module):
    def __init__(self):
        super(RanZcrNet, self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(1, 3, kernel_size=1, bias=False),
            )
        self.resnet= models.resnet34(pretrained=True)
        self.classifier = nn.Sequential(
            nn.Linear(1000, 100),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(100, 10),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(10, 2),
        )

    def forward(self, x):
        x=self.conv1(x)
        x=self.resnet(x)
        output=self.classifier(x)
        return output


model_list = []
for targetCol in range(11):
    model_list.append(torch.load('../input/resnet50/ranzcr_resnet50_col'+str(targetCol)))

valid_dataset = RanzcrDataset('../input')
valid_loader = Data.DataLoader(valid_dataset, batch_size = 16, shuffle = False, num_workers = 4)
criterion = nn.CrossEntropyLoss()

prediction = valid_model(model_list, valid_loader, targetCol, criterion)

df = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/sample_submission.csv")
for i in range(11):
    df[ df.columns[i+1] ] = prediction[i]
df.to_csv("submission.csv", index=False)
print("OVER")